In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy.stats import pearsonr
import torchbnn as bnn

In [2]:
#import creep data
creep_df = pd.read_csv('Ni_superalloys_dataset.csv')
creep_df

,Ni,Al,Co,Cr,Mo,Re,Ru,Ta,W,Ti,Nb,T,log_stress,log_creep_life
0,62.80,5.6,9.0,6.5,0.6,3.0,0.0,6.5,6.0,0.0,0.0,950,2.267172,3.276554
1,59.30,5.8,5.8,2.9,3.9,4.9,6.0,5.6,5.8,0.0,0.0,1100,2.136721,3.026370
2,59.80,5.6,5.6,4.6,2.4,6.4,5.0,5.6,5.0,0.0,0.0,1000,2.389166,3.009026
3,59.30,5.8,5.8,2.9,3.9,4.9,6.0,5.6,5.8,0.0,0.0,1000,2.389166,2.969556
4,61.68,6.0,9.0,3.5,1.5,4.0,0.0,8.0,6.0,0.2,0.0,1100,2.079181,2.957607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,61.00,5.6,9.0,4.3,2.0,2.0,0.0,7.5,8.0,0.0,0.5,1100,2.322219,1.155336
149,61.00,5.6,9.0,4.3,2.0,2.0,0.0,7.5,8.0,0.0,0.5,1070,2.447158,1.089905
150,61.00,5.6,9.0,4.3,2.0,2.0,0.0,7.5,8.0,0.0,0.5,1100,2.352183,0.991226
151,61.00,5.6,9.0,4.3,2.0,2.0,0.0,7.5,8.0,0.0,0.5,1100,2.342423,0.968483


In [3]:
rm_state = 123
test_size = 0.2

X, X_test, y, y_test = train_test_split(np.array(creep_df.iloc[:, 0:13]), np.array(creep_df.iloc[:,13]), shuffle=True, test_size=test_size, random_state=rm_state)

idx = np.arange(len(y))

In [4]:
scaler = MinMaxScaler()

X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [5]:
train_ratio = 0.1

X_train, _, y_train, _, idx_train, idx_pool = train_test_split(X, y, idx, train_size=train_ratio, shuffle=True, random_state=rm_state)

In [6]:
X = X.astype(dtype=np.float32)
X_test =X_test.astype(np.float32)
X_train =X_train.astype(np.float32)

y = y.astype(dtype=np.float32)
y_test = y_test.astype(dtype=np.float32)
y_train = y_train.astype(dtype=np.float32)

X = torch.from_numpy(X)
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
y_train = torch.from_numpy(y_train)
y = torch.from_numpy(y)

y_train = torch.unsqueeze(y_train, dim=1)
y_test = torch.unsqueeze(y_test, dim=1)
y = torch.unsqueeze(y, dim=1)

In [7]:
n_iter = 13
pcc_variance = []
r2_variance = []
rmse_variance = []
mae_variance = []
pcc_random = []
r2_random = []
num_training_data = []

X_train_var = X_train
X_train_ran = X_train
y_train_var = y_train
y_train_ran = y_train
idx_pool_var = idx_pool
idx_pool_ran = idx_pool
idx_train_var = idx_train
idx_train_ran = idx_train

#build the model 
model = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=13, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=1),
)

mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01
optimizer = optim.SGD(model.parameters(), lr=0.001, nesterov=True, momentum=0.95)

for i in range(n_iter):
    print(f"Performing iteration : {i}")

    if i != 0:
        # find 10 data points with the highest variance
        print()
   
        q_points_var = np.argpartition(y_pred_unc_pool_var, -8)[-8:]
        # indices of those points in idx_pool
        idx_pool_train_var = idx_pool_var[q_points_var]

        idx_train_var = np.append(idx_train_var, idx_pool_train_var)
        idx_pool_var = np.delete(idx_pool_var, q_points_var)
        X_train_var = X[idx_train_var]
        y_train_var = y[idx_train_var]

    print(f"Number of training data with variance: {len(idx_train_var)}")
    print(f"Number of pooling data with variance: {len(idx_pool_var)}")
    
    num_training_data.append(len(idx_train_var))

    for step in range(3000):
        y_pred_var = model(X_train_var)
        
        mse = mse_loss(y_pred_var, y_train_var)
        kl = kl_loss(model)
        cost = mse + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if step % 1000==0:
        print('MSE : %2.2f, KL : %2.2f' % (mse.item(), kl.item()))

    n_samples = 1000

    # compute predictions
    y_preds_var = [model(X_test).clone().detach().numpy() for _ in range(n_samples)]
    y_preds_var = np.array(y_preds_var)
    # mean and standard deviation
    y_pred_test_var = np.mean(y_preds_var, axis=0)
    y_pred_unc_test_var = np.std(y_preds_var, axis=0)
    # y_test = y_test.detach().numpy()


    # get the idx pool var
    y_preds_pool_var = [model(X[idx_pool_var]).clone().detach().numpy() for _ in range(n_samples)]
    y_preds_pool_var = np.array(y_preds_pool_var)
    y_pred_unc_pool_var = np.squeeze(np.std(y_preds_pool_var, axis=0))

    print('PCC_test_score', pearsonr(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))[0])
    print('R2_test_score', r2_score(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var)))
    print('RMSE', np.sqrt(mean_squared_error(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))))
    print('MAE', np.mean(abs(np.squeeze(y_test.numpy()) - np.squeeze(y_pred_test_var))))

    pcc_variance.append(pearsonr(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))[0])
    r2_variance.append(r2_score(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var)))
    rmse_variance.append(np.sqrt(mean_squared_error(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))))
    mae_variance.append(np.mean(abs(np.squeeze(y_test.numpy())- np.squeeze(y_pred_test_var))))

Performing iteration : 0
Number of training data with variance: 12
Number of pooling data with variance: 110
PCC_test_score 0.1786703148586526
R2_test_score -1.879705908893786
RMSE 0.59290475
MAE 0.38907248
Performing iteration : 1

Number of training data with variance: 20
Number of pooling data with variance: 102
PCC_test_score 0.14358931785906826
R2_test_score -1.5303146167145307
RMSE 0.5557739
MAE 0.3662951
Performing iteration : 2

Number of training data with variance: 28
Number of pooling data with variance: 94
PCC_test_score 0.486561203007658
R2_test_score 0.030241312457769176
RMSE 0.34406677
MAE 0.25744224
Performing iteration : 3

Number of training data with variance: 36
Number of pooling data with variance: 86
PCC_test_score 0.5586161883007214
R2_test_score 0.26012269652014464
RMSE 0.300532
MAE 0.23046476
Performing iteration : 4

Number of training data with variance: 44
Number of pooling data with variance: 78
PCC_test_score 0.68551390222266
R2_test_score 0.44809924137878

'\n    # random sampling\n    if i != 0:\n        # select 10 random data points\n        q_points_ran = np.random.choice(np.arange(len(idx_pool_ran)), size=10)\n        # indices of those points in idx_pool\n        idx_pool_train_ran = idx_pool_ran[q_points_ran]\n\n        idx_train_ran = np.append(idx_train_ran, idx_pool_train_ran)\n        idx_pool_ran = np.delete(idx_pool_ran, q_points_ran)\n        X_train_ran = X[idx_train_ran]\n        y_train_ran = y[idx_train_ran]\n\n    print(f"Number of training data with random: {len(idx_train_ran)}")\n    print(f"Number of pooling data with random: {len(idx_pool_ran)}")\n\n    for step in range(3000):\n\n\n        y_pred_ran = model(X_train_ran)\n        mse = mse_loss(y_pred_ran, y_train_ran)\n        kl = kl_loss(model)\n        cost = mse + kl_weight*kl\n\n        optimizer.zero_grad()\n        cost.backward()\n        optimizer.step()\n\n    if step % 1000==0:\n        print(\'MSE : %2.2f, KL : %2.2f\' % (mse.item(), kl.item()))\n\n  

In [8]:
import pickle

In [9]:
with open('AL_BNN_VarInf_Ni.pkl', 'wb') as f:
    pickle.dump({'train_numbs':num_training_data, 'pcc':pcc_variance,'r2':r2_variance, 'rsme': rmse_variance, 'mae': mae_variance}, f)
    f.close()

In [10]:
pkl_file = open('AL_BNN_VarInf_Ni.pkl', 'rb')  
test = pickle.load(pkl_file)